In [ ]:
!pip install datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
pip install sumy


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.3 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=bf7673c3d4d00e8ce10f12365593f5289950cab43c7a10967b582c9550b30c0b
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=ea93a49ce3776e5c0a4a4b9ca698b7e4a1d525e1c4b9ae00f19a35798d7ec6b6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
import logging
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Download NLTK resources
nltk.download('punkt')

# Load the SciTLDR dataset
scitldr = load_dataset("allenai/scitldr")

# Preprocess function
prefix = "summarize: "

def preprocess_function(examples):
    try:
        # Convert lists to strings
        source_str = " ".join(examples["source"])
        target_str = " ".join(examples["target"])

        # Add prefix to source
        inputs = prefix + source_str

        # Tokenize inputs
        model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

        # Tokenize labels
        labels = tokenizer(target_str, max_length=512, truncation=True, padding="max_length")

        # Add labels to model_inputs
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except Exception as e:
        logging.error(f"Error in preprocessing function: {e}")
        return None

# Function to apply TextRank summarization
def textrank_summarization(text):
    try:
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = TextRankSummarizer()
        summary = summarizer(parser.document, 3)  # Get the top 3 sentences
        return " ".join([str(sentence) for sentence in summary])
    except ValueError as e:
        logging.error(f"Error in TextRank summarization: {e}")
        return ""

# Apply TextRank summarization to the dataset
def apply_textrank(examples):
    try:
        source_str = " ".join(examples["source"])
        summary = textrank_summarization("In the realm of deep learning, the intricacies of training deep neural networks are profoundly influenced by the choice of optimization algorithms, which in turn govern the convergence behavior of the model. The stochastic gradient descent (SGD) algorithm, and its variants such as Adam and RMSprop, have become the cornerstone of modern deep learning frameworks due to their ability to efficiently traverse the high-dimensional, non-convex loss landscapes that characterize deep models. However, these optimizers are not without their pitfalls. The phenomenon of saddle points, where the gradient vanishes in some directions while being non-zero in others, poses significant challenges by potentially stalling the training process. Furthermore, the introduction of batch normalization and other normalization techniques has been shown to mitigate issues related to internal covariate shift, thereby accelerating convergence and enhancing generalization. Yet, these benefits are counterbalanced by the complexities they introduce, such as the need for careful hyperparameter tuning and the potential for introducing dependencies between training samples. As such, the interplay between optimization strategies and model architecture necessitates a nuanced understanding of both theoretical principles and empirical practices to achieve state-of-the-art performance in deep learning tasks.")
        return {"summary": summary}
    except Exception as e:
        logging.error(f"Error applying TextRank: {e}")
        return {"summary": ""}

try:
    summarized_scitldr = scitldr.map(apply_textrank)
except Exception as e:
    logging.error(f"Error mapping dataset: {e}")

# Display some summarized examples
print(summarized_scitldr['train'][0]['summary'])
print(summarized_scitldr['train'][1]['summary'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


scitldr.py:   0%|          | 0.00/7.21k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/7.56k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

The repository for allenai/scitldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/scitldr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1992 [00:00<?, ? examples/s]

Map:   0%|          | 0/618 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

In the realm of deep learning, the intricacies of training deep neural networks are profoundly influenced by the choice of optimization algorithms, which in turn govern the convergence behavior of the model. The stochastic gradient descent (SGD) algorithm, and its variants such as Adam and RMSprop, have become the cornerstone of modern deep learning frameworks due to their ability to efficiently traverse the high-dimensional, non-convex loss landscapes that characterize deep models. Yet, these benefits are counterbalanced by the complexities they introduce, such as the need for careful hyperparameter tuning and the potential for introducing dependencies between training samples.
In the realm of deep learning, the intricacies of training deep neural networks are profoundly influenced by the choice of optimization algorithms, which in turn govern the convergence behavior of the model. The stochastic gradient descent (SGD) algorithm, and its variants such as Adam and RMSprop, have become 